In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report
import math
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
from torch import Tensor
import torch.nn as nn
import torch.optim as optim

In [2]:
!unzip tf/data/test_data.zip

unzip:  cannot find or open tf/data/test_data.zip, tf/data/test_data.zip.zip or tf/data/test_data.zip.ZIP.


In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [4]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N] - 1

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [5]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k]
        self.length = len(x)

        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [6]:
root_train = '../data/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training'
root_test = '../data/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing'
train_data_path = root_train + '/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_data_path1 = root_test + '/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_data_path2 = root_test + '/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_data_path3 = root_test + '/Test_Dst_NoAuction_ZScore_CF_9.txt'


In [7]:
dec_data = np.loadtxt(train_data_path)
dec_test1 = np.loadtxt(test_data_path1)
dec_test2 = np.loadtxt(test_data_path2)
dec_test3 = np.loadtxt(test_data_path3)

dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 139587)


In [8]:
batch_size = 32

dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

torch.Size([203701, 100, 40]) torch.Size([203701])


In [9]:
for x in dataset_train.x:
    if x.shape == [100, 40]:
        print(x.shape)

In [10]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([32, 100, 40]) torch.Size([32])


In [11]:
import torch
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from model import ViT

# class PreNorm(nn.Module):
#     def __init__(self, dim, fn):
#         super().__init__()
#         self.norm = nn.LayerNorm(dim)  # 正则化
#         self.fn = fn  # 具体的操作
#     def forward(self, x, **kwargs):
#         return self.fn(self.norm(x), **kwargs)

# class FeedForward(nn.Module):
#     def __init__(self, dim, hidden_dim, dropout = 0.):
#         super().__init__()
#         # 前向传播
#         self.net = nn.Sequential(
#             nn.Linear(dim, hidden_dim),
#             nn.GELU(),
#             nn.Dropout(dropout),
#             nn.Linear(hidden_dim, dim),
#             nn.Dropout(dropout)
#         )
#     def forward(self, x):
#         return self.net(x)

# class Attention(nn.Module):
#     # attention
#     def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
#         super().__init__()
#         inner_dim = dim_head *  heads  # 计算最终进行全连接操作时输入神经元的个数
#         project_out = not (heads == 1 and dim_head == dim)  # 多头注意力并且输入和输出维度相同时为True

#         self.heads = heads  # 多头注意力中“头”的个数
#         self.scale = dim_head ** -0.5  # 缩放操作，论文 Attention is all you need 中有介绍

#         self.attend = nn.Softmax(dim = -1)  # 初始化一个Softmax操作
#         self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)  # 对Q、K、V三组向量先进性线性操作

#         # 线性全连接，如果不是多头或者输入输出维度不相等，进行空操作
#         self.to_out = nn.Sequential(
#             nn.Linear(inner_dim, dim),
#             nn.Dropout(dropout)
#         ) if project_out else nn.Identity()

#     def forward(self, x):
#         b, n, _, h = *x.shape, self.heads  # 获得输入x的维度和多头注意力的“头”数
#         qkv = self.to_qkv(x).chunk(3, dim = -1)  # 先对Q、K、V进行线性操作，然后chunk乘三三份
#         q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)  # 整理维度，获得Q、K、V

#         dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale  # Q, K 向量先做点乘，来计算相关性，然后除以缩放因子

#         attn = self.attend(dots)  # 做Softmax运算

#         out = einsum('b h i j, b h j d -> b h i d', attn, v)  # Softmax运算结果与Value向量相乘，得到最终结果
#         out = rearrange(out, 'b h n d -> b n (h d)')  # 重新整理维度
#         return self.to_out(out)  # 做线性的全连接操作或者空操作（空操作直接输出out）

# class Transformer(nn.Module):
#     def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
#         super().__init__()
#         self.layers = nn.ModuleList([])  # Transformer包含多个编码器的叠加
#         for _ in range(depth):
#             # 编码器包含两大块：自注意力模块和前向传播模块
#             self.layers.append(nn.ModuleList([
#                 PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),  # 多头自注意力模块
#                 PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))  # 前向传播模块
#             ]))
#     def forward(self, x):
#         for attn, ff in self.layers:
#             # 自注意力模块和前向传播模块都使用了残差的模式
#             x = attn(x) + x
#             x = ff(x) + x
#         return x

# class ViT(nn.Module):
#     def __init__(self, *, num_classes=3, dim=256, depth=6, heads=8, mlp_dim=512, pool = 'cls', channels = 3, dim_head = 32, dropout = 0.1, emb_dropout = 0.1):
#         super().__init__()
#         num_patches = 100  # 获取图像切块的个数
#         patch_dim = 40  # 线性变换时的输入大小，即每一个图像宽、高、通道的乘积
#         assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'  # 池化方法必须为cls或者mean

#         self.to_patch_embedding = nn.Sequential(
#             nn.Linear(patch_dim, dim),  # 对分割好的图像块进行线性处理（全连接），输入维度为每一个小块的所有像素个数，输出为dim（函数传入的参数）
#         )

#         self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))  # 位置编码，获取一组正态分布的数据用于训练
#         self.cls_token = nn.Parameter(torch.randn(1, 1, dim))  # 分类令牌，可训练
#         self.dropout = nn.Dropout(emb_dropout)

#         self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)  # Transformer模块

#         self.pool = pool
#         self.to_latent = nn.Identity()  # 占位操作

#         self.mlp_head = nn.Sequential(
#             nn.LayerNorm(dim),  # 正则化
#             nn.Linear(dim, num_classes)  # 线性输出
#         )

#     def forward(self, x):
#         x = self.to_patch_embedding(x)  # 切块操作，shape (b, n, dim)，b为批量，n为切块数目，dim为最终线性操作时输入的神经元个数
#         b, n, _ = x.shape  # shape (b, n, 1024)

#         cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)  # 分类令牌，将self.cls_token（形状为1, 1, dim）赋值为shape (b, 1, dim)
#         x = torch.cat((cls_tokens, x), dim=1)  # 将分类令牌拼接到输入中，x的shape (b, n+1, 1024)
#         x += self.pos_embedding[:, :(n + 1)]  # 进行位置编码，shape (b, n+1, 1024)
#         x = self.dropout(x)

#         x = self.transformer(x)  # transformer操作

#         x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

#         x = self.to_latent(x)
#         return self.mlp_head(x)  # 线性输出


In [12]:
model = ViT().to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [14]:
# A function to encapsulate the training loop
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
            print(inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            # print("about to get model output")
            outputs = model(inputs)
            # print("done getting model output")
            # print("outputs.shape:", outputs.shape, "targets.shape:", targets.shape)
            loss = criterion(outputs, targets)
            # Backward and optimize
            # print("about to optimize")
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64) 
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            torch.save(model, './best_val_model_pytorch')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')

    return train_losses, test_losses


In [15]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

...

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([

  0%|          | 0/20 [00:32<?, ?it/s]

torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])
torch.Size([32, 100, 40])


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()

In [9]:
model = torch.load('best_val_model_pytorch')

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = F.softmax(model(inputs), dim=1)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

ModuleNotFoundError: No module named 'vit'

In [ ]:

# model = torch.load('best_val_model_pytorch')
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = F.softmax(model(inputs), dim=1)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)

In [ ]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))


In [ ]:
np.savetxt('train_losses_vit0.01_10.txt', train_losses)
np.savetxt('test_losses_vit0.01_k10.txt', val_losses)